In [36]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Activation, Softmax, MaxPooling2D 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
import torch
import os
import sys
import random
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split

In [37]:
tf.__version__

'2.10.0'

In [38]:
# gpus = tf.config.experimental.list_physical_devices("GPU")
# if gpus:
#     # Restrict TensorFlow to only use the first GPU
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, False)
#             tf.config.experimental.set_virtual_device_configuration(
#                 gpu,
#                 [
#                     tf.config.experimental.VirtualDeviceConfiguration(
#                         memory_limit=4096  # set your limit
#                     )
#                 ],
#             )
#         tf.config.experimental.set_visible_devices(gpus[0], "GPU")
#         logical_gpus = tf.config.experimental.list_logical_devices("GPU")
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#     except RuntimeError as e:
#         # Visible devices must be set before GPUs have been initialized
#         print(e)

In [39]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

if len(physical_devices) == 0:
    print("No GPU devices found. Using CPU.")
else:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


# Data Pre-Processing

Open **dan_train.csv** file and split the games into a list.
Every row of csv: `DL0000000001,B,B[pd],W[dp],B[pp],W[dc],B[de],...`. 

Columns are:

    1. DL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [40]:
# df = open('./Training Dataset/dan_train.csv').read().splitlines()
# gm = [i.split(',',1)[-1] for i in df]
# for i in range(len(gm)):
#     if gm[i][0] != gm[i][-5]:
#         gm[i] = gm[i][0:-6]
# games = [i.split(',',1)[-1] for i in gm]
# print(games[0])
# random.shuffle(games)

In [41]:
df = open('./Training Dataset/dan_train.csv').read().splitlines()
# df = open('./CSVs/Tutorial_dan_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
random.shuffle(games)


Create a dictionary to convert the coordinates from characters to numbers

In [42]:
avg = 0
mn = 12345
mx = 0
count = 0
zeros = 0
for g in games:
    moves_list = g.split(',')
    if len(moves_list) <= 10:
        zeros += 1
    avg += len(moves_list)-1
    mn = min(mn, len(moves_list)-1)
    mx = max(mx, len(moves_list)-1)
    count += 1
print(avg/len(games), mx, mn, zeros)


227.16873003194888 379 107 0


In [43]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates


{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0
 
Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

## Feature engineering

In [44]:
def calculate_liberties(board):
    board[:, :, 4] = np.zeros((19,19)) # initialize

    for x in range(19):
        for y in range(19):
            if board[x, y, 2] == 0:  # 如果是空點，計算相鄰的空點數
                for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < 19 and 0 <= ny < 19:
                        if board[nx, ny, 2]:
                            board[nx, ny, 4] += 1 # liberties += 1

def dfs(x, i, j, visited, stack):
    stack.append((i,j))
    visited[i][j] = 1
    liberty = x[i, j, 4]
    color = 1
    if x[i, j, 0]:
        color = 0

    for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
        nx, ny = i + dx, j + dy
        if 0 <= nx < 19 and 0 <= ny < 19 and x[nx, ny, color] and not visited[nx][ny]:
            liberty += dfs(x, nx, ny, visited, stack)

    return liberty    
    

def calculate_connected(x):
    visited = [[0] * 19 for _ in range(19)]

    for i in range(19):
        for j in range(19):
            if not visited[i][j] and x[i, j, 2]:
                stack = []
                sum = dfs(x, i, j, visited, stack)
                while stack: # while not empty
                    row, col = stack.pop()
                    x[row, col, 4] = sum

def check_dead(x):
    for i in range(19):
        for j in range(19):
            if x[i, j, 4] == 0 and ( x[i, j, 0] == 1 or x[i, j, 1] == 1 ):
                x[i, j, 0] = 0
                x[i, j, 1] = 0
                x[i, j, 2] = 0
                print("it works")


### 5-layer version

In [45]:

# def prepare_input(moves):
#     x = np.zeros((19,19,5))
#     for move in moves:
#         color = (0 if move[0] == 'B' else 1)
#         column = coordinates[move[2]]
#         row = coordinates[move[3]]
#         if color == 0:
#             x[row,column,0] = 1
#             x[row,column,2] = 1
#         if color == 1:
#             x[row,column,1] = 1
#             x[row,column,2] = 1
#     if moves:
#         last_move_column = coordinates[moves[-1][2]]
#         last_move_row = coordinates[moves[-1][3]]
#         calculate_liberties(x)
#         calculate_connected(x)
#         check_dead(x)
#         calculate_liberties(x)
#         calculate_connected(x)
#         x[last_move_row,last_move_column,3] = 1
#     x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
#     return x

# def prepare_label(move):
#     column = coordinates[move[2]]
#     row = coordinates[move[3]]
#     return column*19+row

### 4-layer version

In [46]:

# def prepare_input(moves):
#     x = np.zeros((19,19,5))
#     for move in moves:
#         color = (0 if move[0] == 'B' else 1)
#         column = coordinates[move[2]]
#         row = coordinates[move[3]]
#         if color == 0:
#             x[row,column,0] = 1
#             x[row,column,2] = 1
#         if color == 1:
#             x[row,column,1] = 1
#             x[row,column,2] = 1
#     if moves:
#         last_move_column = coordinates[moves[-1][2]]
#         last_move_row = coordinates[moves[-1][3]]
#         calculate_liberties(x)
#         calculate_connected(x)
#         check_dead(x)
#         x[last_move_row,last_move_column,3] = 1
#     x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
#     return x[:,:,:4]

# def prepare_label(move):
#     column = coordinates[move[2]]
#     row = coordinates[move[3]]
#     return column*19+row

### two layer version

In [47]:
# def prepare_input(moves):
#     x = np.zeros((19,19,2))
#     for move in moves:
#         color = move[0]
#         column = coordinates[move[2]]
#         row = coordinates[move[3]]
#         if color == 'B':
#             x[row,column,0] = 1
#         if color == 'W':
#             x[row,column,0] = 2
#     if moves:
#         last_move_column = coordinates[moves[-1][2]]
#         last_move_row = coordinates[moves[-1][3]]
#         x[last_move_row,last_move_column,1] = 1
#     return x

# def prepare_label(move):
#     column = coordinates[move[2]]
#     row = coordinates[move[3]]
#     return column*19+row

### normal version

In [48]:
def prepare_input(moves):
    x = np.zeros((19,19,4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[last_move_row,last_move_column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    if len(move) < 2:
        print(move)
        return 0
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [49]:
# import cv2

# def prepare_input(moves):
#     x = np.zeros((19,19,3))
#     for move in moves:
#         color = move[0]
#         column = coordinates[move[2]]
#         row = coordinates[move[3]]
#         if color == 'B':
#             x[row,column,0] = 1
#         if color == 'W':
#             x[row,column,0] = 2
#     if moves:
#         last_move_column = coordinates[moves[-1][2]]
#         last_move_row = coordinates[moves[-1][3]]
#         x[last_move_row,last_move_column,1] = 1
#     x = np.pad(x, ((102, 103), (102, 103), (0, 0)), mode='constant')
#     x = cv2.resize(x, (224, 224), interpolation=cv2.INTER_LINEAR)
#     return x

# def prepare_label(move):
#     column = coordinates[move[2]]
#     row = coordinates[move[3]]
#     return column*19+row

The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

Dataset splitting: 90% Training, 10% validation

# Training

### Simple DCNN Model:

In [50]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import BatchNormalization

In [51]:
# !taskkill /F /PID 10952

In [52]:
%load_ext tensorboard
%tensorboard --logdir=path/to/logs --host localhost


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 8564), started 2 days, 7:57:25 ago. (Use '!kill 8564' to kill it.)

In [53]:
def create_model():
    inputs = Input(shape=(19, 19, 4))

    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(inputs)
    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
    outputs = Flatten()(outputs)
    outputs = Dense(256, activation='relu')(outputs)
    outputs = Dropout(0.3)(outputs)
    outputs = Dense(361, activation='softmax')(outputs)  # Adjust the number of units based on your output space
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [54]:
def create_model1():
    inputs = Input(shape=(19, 19, 4))
    
    x = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)  # Add BatchNormalization
    x = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(x)

    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.6)(x)
    policy_head = Dense(361, activation='softmax', name='policy')(x)
    value_head = Dense(1, activation='tanh', name='value')(x)

    model = Model(inputs, [policy_head, value_head])

    opt = Adam(learning_rate=0.001)

    model.compile(optimizer=opt,
                  loss={'policy': 'categorical_crossentropy', 'value': 'mean_squared_error'},
                  metrics={'policy': 'accuracy', 'value': 'mae'})
    
    return model


In [55]:
def create_model2():
    inputs = Input(shape=(19, 19, 4))

    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    
    outputs = Flatten()(outputs)
    outputs = Dense(256, activation='relu')(outputs)
    outputs = Dropout(0.5)(outputs)
    outputs = Dense(361, activation='softmax')(outputs)  # Adjust the number of units based on your output space
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.001)  # Adjust the learning rate
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',  # Change to sparse_categorical_crossentropy
                  metrics=['accuracy'])

    return model

In [56]:
from tensorflow.keras.applications import ResNet50

def create_ResNetmodel(img_shape, num_classes):
    model_resnet = ResNet50(include_top=False,weights='imagenet')
    # Make resnet50 model layers as non trainable
    for layer in model_resnet.layers:
        layer.trainable = False
    img_input = Input(shape=img_shape)
    img_model_resnet = model_resnet(img_input)
    x = Flatten(name='flatten')(img_model_resnet)
    x = Dense(256, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    modified_model = Model(inputs=img_input, outputs=x)
    modified_model.compile(loss='sparse_categorical_crossentropy',
    optimizer='adam', metrics=['acc'])
    return modified_model

In [57]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Input

def basic_block(x, filters, stride=1):
    identity = x
    x = Conv2D(filters, kernel_size=(3, 3), strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    if stride != 1:
        identity = Conv2D(filters, kernel_size=(1, 1), strides=stride, padding='same')(identity)
        identity = BatchNormalization()(identity)
    x = Add()([x, identity])
    x = Activation('relu')(x)
    return x

def resnet34(input_shape=(224, 224, 3)):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, kernel_size=(7, 7), strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Stack basic blocks (adjust the number of blocks as needed)
    x = basic_block(x, filters=64)
    x = basic_block(x, filters=64)
    x = basic_block(x, filters=64)

    x = basic_block(x, filters=128, stride=2)
    x = basic_block(x, filters=128)
    x = basic_block(x, filters=128)
    x = basic_block(x, filters=128)

    x = basic_block(x, filters=256, stride=2)
    x = basic_block(x, filters=256)
    x = basic_block(x, filters=256)
    x = basic_block(x, filters=256)
    x = basic_block(x, filters=256)
    x = basic_block(x, filters=256)

    x = basic_block(x, filters=512, stride=2)
    x = basic_block(x, filters=512)
    x = basic_block(x, filters=512)

    # Add global average pooling and output layer
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = Dense(361, activation='softmax')(x)  # Adjust num_classes

    model = tf.keras.Model(inputs, outputs)

    opt = Adam(learning_rate=0.001)  # Adjust the learning rate
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',  # Change to sparse_categorical_crossentropy
                  metrics=['accuracy'])
                  
    return model

In [58]:
# Usage example:
model = create_model2()
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 19, 19, 4)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 19, 19, 64)        12608     
                                                                 
 batch_normalization_6 (Batc  (None, 19, 19, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_7 (Conv2D)           (None, 19, 19, 64)        200768    
                                                                 
 batch_normalization_7 (Batc  (None, 19, 19, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_8 (Conv2D)           (None, 19, 19, 64)        1024

In [59]:
# model = create_model2()
# # model.summary()


## data generator

In [60]:
from keras.backend import set_session
from keras.backend import clear_session
from keras.backend import get_session
from tensorflow.keras.callbacks import TensorBoard
import gc
import math
import time


In [61]:

# Reset keras function from: https://github.com/keras-team/keras/issues/12625
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    # try:
    #     del classifier # this is from global space - change this as you need
    # except:
    #     pass
    gc.collect()
    # print("Garbage Collected: " + gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=1, visible_device_list="0")
    config = tf.compat.v1.ConfigProto(gpu_options=gpu_options)
    set_session(tf.compat.v1.Session(config=config))


In [62]:
# Assuming prepare_input and prepare_label functions are defined elsewhere

def data_generator(games, batch_size):
    for batch_start in range(0, len(games), batch_size):

        batch_end = batch_start + batch_size
        batch_games = games[batch_start:batch_end]
        x = []
        y = []
        for game in batch_games:
            moves_list = game.split(',')
            for count, move in enumerate(moves_list):
                x.append(prepare_input(moves_list[:count]))
                y.append(prepare_label(moves_list[count]))

        x = np.array(x)
        y = np.array(y)
        # batch_number = batch_start/batch_size
        # np.save(f'inputs/x{batch_number}.npy', x)
        # np.save(f'inputs/y{batch_number}.npy', y)

        

        # x = np.load(f'inputs/x{batch_number}.npy')
        # y = np.load(f'inputs/y{batch_number},npy')
        
        y_one_hot = tf.one_hot(y, depth=19*19)

        x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)

        yield (x_train, y_train, x_val, y_val)


In [63]:
# class MCTSNode:
#     def __init__(self, state):
#         self.state = state
#         self.parent = None
#         self.children = {}
#         self.visits = 0
#         self.score = 0

# def mcts_predict(board_states):
#     root = MCTSNode(board_states)  # 初始化 MCTS 根節點
#     number_of_simulation = 500
#     EXPLORATION_CONST = 5
    
#     # MCTS 搜索
#     for i in range(number_of_simulation):
#         node = root
#         state = board_states.copy()  # 假設有一個 copy 方法來複製棋盤狀態
#         # 選擇節點，根據 UCB 算法
#         while node.children:
#             # UCB1 公式
#             child = max(node.children.items(), key=lambda c: c[1].score / (c[1].visits + 1.0) + EXPLORATION_CONST * np.sqrt(np.log(node.visits + 1.0) / (c[1].visits + 1.0)))
#             state = child[0]  # 假設這是下一步的狀態
#             node = child[1]
        
#         # 展開節點，根據 MCTS 算法
#         if not state.is_game_over():
#             legal_moves = state.get_legal_moves()  # 假設有一個 get_legal_moves 方法來獲取合法走法
#             for move in legal_moves:
#                 new_state = move  # 假設這是一個新的棋盤狀態
#                 node.children[move] = MCTSNode(new_state)
        
#         # 模擬遊戲
#         rollout_state = state.copy()
#         while not rollout_state.is_game_over():
#             legal_moves = rollout_state.get_legal_moves()
#             move = random.choice(legal_moves)
#             rollout_state = move  # 假設這是下一步的狀態
        
#         # 回溯更新節點
#         while node is not None:
#             node.visits += 1
#             node.score += rollout_state.get_score()  # 假設有一個 get_score 方法來獲取當前狀態得分
#             node = node.parent
    
#     # 根據 MCTS 搜索結果返回預測的下一步棋
#     best_child = max(root.children.items(), key=lambda c: c[1].visits)
#     return best_child[0]


In [64]:
# batch_size = 512  # 調整批次大小
# batchs = math.ceil(len(games) / batch_size)

# # 設置 TensorBoard 回調
# tensorboard_callback = TensorBoard(log_dir=f"dan_logs/{time.time()}", histogram_freq=1)

# for epoch in range(1, 2):
#     print("epoch", epoch)
#     batch_count = 1
    
#     for x_train, y_train, x_val, y_val in data_generator(games, batch_size):
#         print(f"{batch_count}/{batchs}")

#         # MCTS 預測下一步棋步
#         next_move = mcts_predict(x_train)

#         # 將 MCTS 預測的棋步轉換為模型所需格式
#         mcts_input = prepare_input(next_move)
#         mcts_label = prepare_label(next_move)

#         # 添加 MCTS 預測的棋步到訓練數據
#         x_train = np.append(x_train, [mcts_input], axis=0)
#         y_train = np.append(y_train, [mcts_label], axis=0)

#         # 訓練模型
#         history = model.fit(
#             x=x_train, 
#             y=y_train,
#             batch_size=1024,
#             epochs=1,
#             validation_data=(x_val, y_val),
#             callbacks=[tensorboard_callback]
#         )

#         if batch_count % 10 == 0:
#             model.save(f"./models/dan_{batch_count}_{history.history['val_accuracy'][0]:.5f}_{history.history['val_loss'][0]:.5f}.h5")

#         batch_count += 1
#         reset_keras()


In [65]:

batch_size = 512  # Adjust this if needed
batchs = math.ceil(len(games) / batch_size)

# Set up TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=f"dan_logs/{time.time()}", histogram_freq=1)

for epoch in range(1, 2):
    print("epoch", epoch)
    batch_count = 1
    
    for x_train, y_train, x_val, y_val in data_generator(games, batch_size):
        print(f"{batch_count}/{batchs}")
        history = model.fit(
            x=x_train, 
            y=y_train,
            batch_size=1024,
            epochs=1,
            validation_data=(x_val, y_val),
            callbacks=[tensorboard_callback]  # Add TensorBoard callback here
        )

        if batch_count % 10 == 0:
            model.save(f"./models/dan2_{batch_count}_{history.history['val_accuracy'][0]:.5f}_{history.history['val_loss'][0]:.5f}.h5")
            # model.save(f"./models/dan_{batch_count}_{history.history['val_policy_accuracy'][0]:.5f}_{history.history['val_loss'][0]:.5f}.h5")

        batch_count += 1
        reset_keras()


epoch 1
(115530, 19, 19, 4)
1/196
102/102 [==============================] - 21s 177ms/step - loss: 5.8406 - accuracy: 0.0056 - val_loss: 5.8699 - val_accuracy: 0.0029
(117565, 19, 19, 4)
2/196
 28/104 [=======>......................] - ETA: 10s - loss: 5.7194 - accuracy: 0.0096

KeyboardInterrupt: 

In [ ]:
print("val_acc:", history.history['val_accuracy'])
model.save(f'./models/model_dan2.h5')

val_acc: [0.47193917632102966]


## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!